In [2]:
# Importing the libraries
import numpy as np
#import matplotlib.pyplot as plt

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout # GRU, Bidirectional
#from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error

In [3]:
data = pd.read_csv('/content/drive/MyDrive/files2/TSLA.csv', index_col='Date', parse_dates=['Date'])
data.head() #view first five columns

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-06-29,3.800,5.000,3.508,4.778,4.778,93831500
2010-06-30,5.158,6.084,4.660,4.766,4.766,85935500
2010-07-01,5.000,5.184,4.054,4.392,4.392,41094000
2010-07-02,4.600,4.620,3.742,3.840,3.840,25699000
2010-07-06,4.000,4.000,3.166,3.222,3.222,34334500


In [4]:
# Convert '2016' and '2017' to datetime objects for slicing
# Assuming 'Date' is a datetime column
mytrain = data[data.index < '2017-01-01'].iloc[:, 1:2].values
mytest = data[data.index >= '2017-01-01'].iloc[:, 1:2].values

In [5]:
# Scaling the training set
sc = MinMaxScaler(feature_range=(0,1))  #MinMaxScaler(feature_range= (start,stop))
mytrain_scaled = sc.fit_transform(mytrain)  #instance.fit_transform(data)

In [6]:
mytrain_scaled #view the scaled values!
#82.55 ==> 0.06065...

array([[0.03045962],
       [0.05018378],
       [0.03380763],
       ...,
       [0.75392114],
       [0.73718112],
       [0.73099458]])

In [7]:
len(mytrain_scaled)

1640

In [8]:
I_train = []
O_train = []
for i in range(60, len(mytrain_scaled)):  # Change the upper bound of the range
    I_train.append(mytrain_scaled[i-60:i,0])
    O_train.append(mytrain_scaled[i,0])

In [9]:
I_train[0]

array([0.03045962, 0.05018378, 0.03380763, 0.02354525, 0.01226391,
       0.        , 0.00323884, 0.00462171, 0.00524037, 0.00731468,
       0.01280978, 0.01772262, 0.0169948 , 0.02045198, 0.01899632,
       0.01553914, 0.01681284, 0.01794097, 0.01772262, 0.0165581 ,
       0.01553914, 0.01546636, 0.01386513, 0.01579388, 0.01936024,
       0.02019724, 0.01790458, 0.01284617, 0.01219113, 0.01099021,
       0.00818807, 0.00462171, 0.00662324, 0.00789694, 0.01008043,
       0.01077186, 0.00953455, 0.00902507, 0.01368318, 0.01120856,
       0.01219113, 0.01324648, 0.01179082, 0.01295535, 0.01149969,
       0.01477492, 0.01677645, 0.0169948 , 0.01590305, 0.0157211 ,
       0.01608501, 0.01564831, 0.01553914, 0.01808654, 0.0195422 ,
       0.0237636 , 0.01706758, 0.01717675, 0.01790458, 0.0157211 ])

In [10]:
I_train[0]

array([0.03045962, 0.05018378, 0.03380763, 0.02354525, 0.01226391,
       0.        , 0.00323884, 0.00462171, 0.00524037, 0.00731468,
       0.01280978, 0.01772262, 0.0169948 , 0.02045198, 0.01899632,
       0.01553914, 0.01681284, 0.01794097, 0.01772262, 0.0165581 ,
       0.01553914, 0.01546636, 0.01386513, 0.01579388, 0.01936024,
       0.02019724, 0.01790458, 0.01284617, 0.01219113, 0.01099021,
       0.00818807, 0.00462171, 0.00662324, 0.00789694, 0.01008043,
       0.01077186, 0.00953455, 0.00902507, 0.01368318, 0.01120856,
       0.01219113, 0.01324648, 0.01179082, 0.01295535, 0.01149969,
       0.01477492, 0.01677645, 0.0169948 , 0.01590305, 0.0157211 ,
       0.01608501, 0.01564831, 0.01553914, 0.01808654, 0.0195422 ,
       0.0237636 , 0.01706758, 0.01717675, 0.01790458, 0.0157211 ])

In [11]:
I_train = np.array(I_train)   #converting into arrays!
O_train = np.array(O_train)

In [12]:
I_train = np.array(I_train)   #converting into arrays!
O_train = np.array(O_train)

In [13]:
I_train.shape #samples,steps

(1580, 60)

In [16]:
# Calculate the appropriate first dimension for reshaping
new_dim1 = I_train.size // (60 * 1)
I_train = I_train.reshape(new_dim1, 60, 1)

# Verify the new shape
print(I_train.shape)

(1580, 60, 1)


In [17]:
I_train.shape

(1580, 60, 1)

In [18]:
#building the model

model = Sequential()
# First LSTM layer
model.add(LSTM(units=50, return_sequences=True, input_shape=(I_train.shape[1],1)))
model.add(Dropout(0.2))
# Second LSTM layer
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# Third LSTM layer
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# Fourth LSTM layer
model.add(LSTM(units=50))
model.add(Dropout(0.2))
# The output layer
model.add(Dense(units=1))

In [19]:
#compile
model.compile(optimizer='rmsprop',loss='mean_squared_error')

In [20]:
model.fit(I_train,O_train,epochs=50,batch_size=32)

Epoch 1/50
50/50 [==============================] - 14s 140ms/step - loss: 0.0270
Epoch 2/50
50/50 [==============================] - 11s 217ms/step - loss: 0.0107
Epoch 3/50
50/50 [==============================] - 9s 184ms/step - loss: 0.0088
Epoch 4/50
50/50 [==============================] - 8s 156ms/step - loss: 0.0083
Epoch 5/50
50/50 [==============================] - 5s 108ms/step - loss: 0.0076
Epoch 6/50
50/50 [==============================] - 5s 103ms/step - loss: 0.0069
Epoch 7/50
50/50 [==============================] - 7s 139ms/step - loss: 0.0065
Epoch 8/50
50/50 [==============================] - 5s 102ms/step - loss: 0.0060
Epoch 9/50
50/50 [==============================] - 7s 137ms/step - loss: 0.0062
Epoch 10/50
50/50 [==============================] - 5s 105ms/step - loss: 0.0057
Epoch 11/50
50/50 [==============================] - 5s 102ms/step - loss: 0.0052
Epoch 12/50
50/50 [==============================] - 7s 140ms/step - loss: 0.0054
Epoch 13/50
50/50 [====

In [24]:
model.save("StockTSLA.h5")

In [26]:
#Test
from tensorflow.keras.models import load_model
import numpy as np
#load the model
model = load_model("/content/StockTSLA.h5")
#60 values
user_input = [83.58746017, 80.34122742, 80.81285462, 82.00291895, 83.29159961, 81.44688209, 80.76357678, 83.80165525, 82.51120409, 82.48569342, 80.65383659, 81.69740582, 82.78338862, 83.37704145, 83.53829805, 80.20374715, 80.42476455, 82.69989097, 83.73417143, 82.95666579, 83.94381372, 82.37856671, 83.60787851, 81.80850007, 82.61239819, 82.06021048, 81.0673498, 83.7835985, 81.74964384, 83.10783276, 80.54146548, 80.49215929, 82.03237911, 81.79802128, 80.93914228, 83.90444224, 83.2143073, 81.31844381, 83.23807452, 83.4603476, 81.85039395, 81.3031393, 80.6351211, 83.65161771, 80.54893536, 82.22940581, 82.50289209, 80.59368723, 83.24620825, 81.77694404, 83.20275117, 83.43996357, 83.38335394, 82.69282925, 80.80822433, 80.30044039, 83.6842126, 82.82960781, 80.12425038, 82.25607533]

#2D array
user_input_array = np.array(user_input).reshape(60, 1)
#down-scale
sc = MinMaxScaler(feature_range=(0,1))
user_scaled = sc.fit_transform(user_input_array)
#3D array
user_scaled = user_scaled.reshape(1, 60, 1)
#predict
pred = model.predict(user_scaled)
print(pred) #down scale output
#up-scale
pred_original = sc.inverse_transform(pred)
print("The stock price is",pred_original[0][0]) #up scaled/original o/p


1/1 [==============================] - 2s 2s/step
[[0.5373447]]
The stock price is 82.176674


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
